In [4]:
#Searching for MaNGA data 
import os
from astropy.io import fits
from astropy.table import Table
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u

In [5]:
# Download DR16 drpall file
# https://data.sdss.org/sas/dr16/manga/spectro/redux/

# Read in DR16
drpall = Table.read('/Users/mmckay/Desktop/research/dr16_drpall-v2_4_3.fits')

t=Table()
t['Objra'] = drpall['objra'] 
t['Objdec'] = drpall['objdec']
t['mangaid'] = drpall['mangaid']
t['plateifu'] = drpall['plateifu']
# Check for duplicated RA and Dec values (True = duplicates)
len(drpall['objra']) != len(set(drpall['objra']))
len(drpall['objdec']) != len(set(drpall['objdec']))

#dr16_ra = np.unique(drpall['objra'])
#dr16_dec = np.unique(drpall['objdec'])

# Input DR16 ra and dec into Skycoord
dr16_skycoord = SkyCoord(ra=drpall['objra'], dec=drpall['objdec'], frame='icrs', unit=u.deg)
dr16_skycoord

<SkyCoord (ICRS): (ra, dec) in deg
    [(194.52342394, 29.01735263), (298.03371   , 27.752152  ),
     (332.60208984, 11.7130773 ), ..., (121.74477921, 46.5629692 ),
     (121.43474063, 46.02442403), ( 70.32185   , 25.525777  )]>

In [3]:
hdu = fits.open('/Users/mmckay/Desktop/research/dr16_drpall-v2_4_3.fits')
hdu[1].header['*']

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1340 / length of dimension 1                          
NAXIS2  =                11465 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   99 / number of table fields                         
TTYPE1  = 'plate   '                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'ifudsgn '                                                            
TFORM2  = '21A     '                                                            
TTYPE3  = 'plateifu'        

In [93]:
# Read in BBRD sample
my_table=np.loadtxt('sdss_129galaxies.txt', usecols=(0,1), skiprows=1)

my_ra_list = np.empty((0,126))
my_dec_list = np.empty((0,1286))
for i in my_table:
    ra_dec = np.split(i,2)
    ra, dec = ra_dec[0], ra_dec[1]
    ra, dec = '%.3f' % float(ra), '%.3f' % float(dec)
    my_ra_list = np.append(my_ra_list, ra)
    my_dec_list = np.append(my_dec_list, dec)

# Check for duplicated RA and Dec values (True = duplicates)
len(my_ra_list) != len(set(my_ra_list))
len(my_dec_list) != len(set(my_dec_list))

# Input DR16 ra and dec into Skycoord
bbrd_skycoord = SkyCoord(ra=my_ra_list, dec=my_dec_list, frame='icrs', unit=u.deg)
bbrd_skycoord


<SkyCoord (ICRS): (ra, dec) in deg
    [(170.862, -0.714), (194.857, -2.248), (121.68 , 45.661),
     (160.746, 65.146), (213.962,  1.883), (173.693,  2.921),
     (121.921, 39.004), (143.092, 55.919), (150.034,  2.765),
     (133.774, 51.635), (131.726,  2.56 ), (165.108,  4.224),
     (168.869,  4.349), (150.415,  4.384), (128.633, 42.916),
     (223.417,  2.921), (135.322, 47.36 ), (175.71 , 59.802),
     (198.292,  5.328), (246.518, 46.368), (238.317, 53.544),
     (241.033, 47.35 ), (247.674, 40.529), (226.835,  2.391),
     (247.304, 41.151), (262.074, 60.015), (128.688, 38.342),
     (171.785, 56.252), (189.817, 59.437), (155.704,  6.796),
     (167.906,  7.412), (202.204, 53.443), (194.536, 54.467),
     (255.248, 24.433), (231.591, 48.704), (127.357, 30.512),
     (162.95 , 10.866), (136.337, 57.16 ), (132.913, 57.107),
     (161.739, 39.562), (198.189, 46.935), (209.888, 56.041),
     (192.769, 48.409), (238.398, 32.306), (233.899, 35.636),
     (155.904, 42.351), (162.59 , 4

In [94]:
match = bbrd_skycoord.match_to_catalog_sky(catalogcoord=dr16_skycoord, nthneighbor=1.)

In [95]:
len(bbrd_skycoord.ra.value)

126

In [96]:
prelim_ra_match_arr = np.empty((0, 126))
prelim_dec_match_arr = np.empty((0, 126))
for idx in match[0]:
    prelim_ra_match = '%.3f' % dr16_skycoord.ra.value[idx]
    prelim_dec_match = '%.3f' % dr16_skycoord.dec.value[idx]
    prelim_ra_match_arr = np.append(prelim_ra_match_arr, prelim_ra_match)
    prelim_dec_match_arr = np.append(prelim_dec_match_arr, prelim_dec_match)
    #print(prelim_ra_match, prelim_dec_match)
    

match_ra = np.empty((0, 126))
match_dec = np.empty((0, 126))
a = 0
for mra, mdec in zip(prelim_ra_match_arr, prelim_dec_match_arr):
    for ra, dec in zip(my_ra_list, my_dec_list):
        if mra == ra: #and mdec == dec:
            a += 1
            print(a,ra,dec)
            match_ra = np.append(match_ra, ra)
            match_dec = np.append(match_dec, dec)
            
            
match_table = Table()
match_table['match_ra'] = match_ra
match_table['match_dec'] = match_dec
match_table.write('dr16_match_ra_dec.txt',format='ascii.basic', overwrite=True)

1 121.921 39.004
2 247.674 40.529
3 247.304 41.151
4 132.913 57.107
5 198.189 46.935
6 163.219 43.428
7 221.438 51.581
